In [7]:
# Selecting libraries
import yfinance as yf
import pandas as pd
import numpy as np
import pyfolio as py
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings("ignore")

import cvxopt as opt
from cvxopt import blas, solvers

In [2]:
# Define the assets and key initial metrics as investment amount to generate a portfolio

stocks = ['SPY', '^TNX', 'TIP', 'GLD', 'DBC' ]  # Assets to select yfinance format
portfolio_value = 10**6                         # Initial Portfolio Value to be allocated in full
weights = [0.30 , 0.15, 0.40, 0.075, 0.075]          # Weight Allocation per asset

benchmark = '^GSPC'                    # Which is your benchmark? ^GSPC is SP500 for Yfinance library

start_date = '2010-01-01'              # Start date for asset data download
live_date = '2020-01-01'               # Portfolio LIVE start date (for analytics)


# Warning handling
if len(weights) != len(stocks):
    print(sum((np.array(weights))))
    print('Number of Asset does not match weight allocated')
    
weight = round(sum((np.array(weights))))
if weight != 1.0:
    print(sum((np.array(weights))))
    print('Weight could not be allocated properly, make sure they sum 1.0')


In [3]:
# Download data, clean and isolate values for calculation
# If you use other data source, make sure the format is the same than stock_data.head() on next cell. Check dtypes by stock_data.info()

stock_data = yf.download(stocks, start=start_date)['Adj Close']
stock_data = stock_data.dropna()
stock_data = stock_data.reindex(columns=stocks)
stock_prices = stock_data[stocks].values

[*********************100%***********************]  5 of 5 completed


In [5]:
shares_df = pd.DataFrame(index=[stock_data.index[0]])

for s,w in zip(stocks, weights):
    shares_df[s + '_shares'] = np.floor((portfolio_value * np.array(w)) / stock_data[s][0])

shares_df    

,SPY_shares,^TNX_shares,TIP_shares,GLD_shares,DBC_shares
2010-01-04,3417.0,39052.0,5330.0,683.0,3076.0


In [13]:
def get_optimal_weights(cov_matrix, expected_returns, num_assets):
    # minimize w'Σw subject to μ'w = 1
    solvers.options['show_progress'] = False

    # Define the 2nd part of the minimization problem
    P = opt.matrix(cov_matrix.values)
    q = opt.matrix(np.zeros((num_assets, 1)))

    # Define the 1st part of the minimization problem
    G = opt.matrix(-np.eye(num_assets))
    h = opt.matrix(np.zeros((num_assets, 1)))

    # Define the equality constraint
    A = opt.matrix(expected_returns.values.reshape((1, -1)))
    b = opt.matrix(1.0)

    # Optimization
    sol = solvers.qp(P, q, G, h, A, b)
    weights = np.array(sol['x'])

    return weights

In [22]:
df = stock_data.pct_change().dropna()
np.isinf(df).values.sum()

0

In [23]:

# Initialize portfolio weights DataFrame
portfolio_weights = pd.DataFrame(columns=stocks, index=stock_data.index)

# Initialize rebalance signal
signal = False

# Initialize previous balance month
balance_month = stock_data.index[0].month

for month_end in stock_data.index:
    # Rebalance at the end of each month
    if month_end.month != balance_month or month_end == stock_data.index[-1]:
        balance_month = month_end.month
        signal = True

        # Define the rolling window (set at 12 months)
        window = stock_data[(stock_data.index < month_end) & (stock_data.index > month_end - pd.DateOffset(months=12))]

        # Calculate returns for the window
        returns = window.pct_change().dropna()
        returns 

        # Calculate expected returns and the covariance matrix
        expected_returns = returns.mean()
        cov_matrix = returns.cov()

        # Get optimal weights using MVO
        optimal_weights = get_optimal_weights(cov_matrix, expected_returns, len(stocks))

        # Assign weights to the portfolio
        portfolio_weights.loc[month_end] = optimal_weights.flatten()
    else:
        # If not the end of the month, the weights remain the same as the previous day
        portfolio_weights.loc[month_end] = portfolio_weights.shift(1).loc[month_end]


ValueError: domain error